## Goals

mencari tahu pengaruh status perkawinan,education,family_status,income type seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam melunasi pinjaman, dan tentunya mencari tahu tujuan mereka meminjam uang dan persentase kegagalan mereka dalam mengembalikan uang tersebut

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
data = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/credit_scoring_eng.csv')

Pra-pemrosesan data

In [3]:
#Mengidentifikasi dan mengisi nilai-nilai yang hilang
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
#data['days_employed'] = data['days_employed'].astype(int)

terlihat ada data yang hilang pada column days_employed dan total_income

**Deskripsi Data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - pengidentifikasi untuk status perkawinan nasabah
- `family_status_id` - tanda pengenal status perkawinan
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah pernah melakukan gagal bayar pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman

In [5]:
#melihat data awal
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [6]:
#melihat data yang nilainya hilang
data_null = data[data['days_employed'].isna() & (data['total_income'].isna())]
data_null

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


data yang yang hilang ada 2174 data

In [7]:
#megidentifikasi disribusi data yang hilang dari income_type apa
a = data_null['income_type'].value_counts().reset_index().rename(columns={'income_type':'total'})
b = data_null['income_type'].value_counts(normalize=True).reset_index()
b['income_type'] = b['income_type'].apply("{:,.2%}".format)
a.merge(b,on='index')

,index,total,income_type
0,employee,1105,50.83%
1,business,508,23.37%
2,retiree,413,19.00%
3,civil servant,147,6.76%
4,entrepreneur,1,0.05%


terlihat dari data yang hilang ternyata yang paling banyak dari income_type adalah employee, dengan menyumbang 50% diikuti business

In [8]:
#mengidentifikasi distribusi income_type seluruh dataset
data['income_type'].value_counts(normalize=True).apply("{:,.2%}".format)

employee                       51.66%
business                       23.62%
retiree                        17.91%
civil servant                   6.78%
unemployed                      0.01%
entrepreneur                    0.01%
student                         0.00%
paternity / maternity leave     0.00%
Name: income_type, dtype: object

In [9]:
data_null

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [10]:
#memeriksa data yang hilang berdasarkan umur (dob_years)
data_null.pivot_table(index='dob_years',columns='income_type',values='debt',aggfunc='count',margins=True)

income_type,business,civil servant,employee,entrepreneur,retiree,All
dob_years,,,,,,
0,2.0,NaN,5.0,NaN,3.0,10
19,NaN,NaN,1.0,NaN,NaN,1
20,1.0,NaN,4.0,NaN,NaN,5
21,7.0,1.0,10.0,NaN,NaN,18
22,6.0,NaN,11.0,NaN,NaN,17
23,5.0,1.0,30.0,NaN,NaN,36
24,9.0,1.0,10.0,NaN,1.0,21
25,4.0,4.0,15.0,NaN,NaN,23
26,9.0,2.0,24.0,NaN,NaN,35


data yang hilang tidak memiliki pola dan murni data yang hiilang adalah secara acak

# transformasi data

### transformasi pada data education

In [11]:
#karena disini tidak sama untuk nama education saya akan mengubahnya menjadi lower case untuk setiap datanya, sekaligus menghapus duplikasi datanya
data['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [12]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

### transformasi pada data anak

In [13]:
#melihat jumlah anak pada dataset
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

terdapat nilai yang aneh disini pada jumlah anak karena ada yang nilainya -1, agar lebih mempermudah analisis saya akan melakukan drop pada datanya

In [14]:
data = data.drop(data[data['children']==-1].index)
data['children'].value_counts()

0     14149
1      4818
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

### transformasi data days_employed

In [15]:
data['days_employed'].describe()

count     19307.000000
mean      63064.804509
std      140845.743155
min      -18388.949901
25%       -2746.531801
50%       -1203.245164
75%        -290.865453
max      401755.400475
Name: days_employed, dtype: float64

disini ada masalah karena nilai days_employed adalah negatif, untuk itu perlu dilakukan transformasi

In [16]:
data['days_employed'] =  data['days_employed'].abs()

In [17]:
data['days_employed'] = data['days_employed']/365

In [18]:
data = data.drop(data[data['days_employed']>100].index)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,11.026860,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,15.406637,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,11.300677,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
5,0,2.537495,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,6.442279,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate
21520,1,12.409087,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21522,1,5.789991,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,8.527347,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


data lama bekerja sudah diubah menjadi tahun, dan tadi ada data yang tidak masuk akal karena lama bekerja sampai dengan 900 tahun maka datanya tidak terpakai dan di drop

memeriksa data nasabah berdasarkan umur (dob) years

In [19]:
data['dob_years'].describe()

count    18040.000000
mean        40.276053
std         10.977319
min          0.000000
25%         32.000000
50%         40.000000
75%         48.000000
max         75.000000
Name: dob_years, dtype: float64

In [20]:
data[data['dob_years']==0].shape

(84, 12)

ada yang aneh yaitu umur nya adalah 0 saat bekerja, karena data ini salah maka akan di drop agar tidak mengganggu analisis

In [21]:
data = data.drop(data[data['dob_years']==0].index)

In [22]:
data['dob_years'].describe()

count    17956.000000
mean        40.464469
std         10.650856
min         19.000000
25%         32.000000
50%         40.000000
75%         48.000000
max         75.000000
Name: dob_years, dtype: float64

### transformasi data family status

In [23]:
#melihat apakah pada data famulty status ada keaneahan atau tidak
data['family_status'].value_counts()

married              10439
civil partnership     3574
unmarried             2485
divorced               985
widow / widower        473
Name: family_status, dtype: int64

tidak ada keanehan data pada family status sehingga tidak perlu transformasi data

### transformasi data gender

In [24]:
data['gender'].value_counts()

F      11344
M       6611
XNA        1
Name: gender, dtype: int64

In [25]:
#walaupun cuma ada 1 data tentang gender yang tidak jelas saya akan melakukan transformasi dengan melakukan drop pada data tidak jelas ini
data = data.drop(data[data['gender']=='XNA'].index)
data['gender'].value_counts()

F    11344
M     6611
Name: gender, dtype: int64

### transformasi data income type

In [26]:
data['income_type'].value_counts()

employee                       11038
business                        5055
civil servant                   1449
retiree                          409
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

terlihat tidak ada keanehan pada data income_type, tidak perlu perubahan lebih lanjut

In [27]:
#cek duplikasi data
data.duplicated().sum()

71

In [28]:
#menghapus data duplicate
data = data.drop_duplicates()
data.duplicated().sum()

0

In [29]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,11.026860,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,15.406637,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,11.300677,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
5,0,2.537495,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,6.442279,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate
21520,1,12.409087,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21522,1,5.789991,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,8.527347,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


### membuat dictionary data guna melihat kategori nilai lebih cepat

menggunakan dictionary untuk melihat kolom yang memiliki id lebih cepat dan kita bisa melakukan kategori pada id tersebut dan bisa menghitung jumlah dari kategori yang sudah ditentukan, ini membantuk untuk melihat jumlah data per-kategori dan melakukan bebrapa fungsi yang nantinya membantu untuk proses analisis data

In [30]:
education_dict = data[['education_id','education']].set_index('education_id').to_dict()['education']
education_dict 

{0: "bachelor's degree",
 1: 'secondary education',
 2: 'some college',
 3: 'primary education',
 4: 'graduate degree'}

In [31]:
status_dict = data[['family_status_id','family_status']].set_index('family_status_id').to_dict()['family_status']
status_dict

{0: 'married',
 1: 'civil partnership',
 3: 'divorced',
 4: 'unmarried',
 2: 'widow / widower'}

### Mengisi nilai yang hilang di total_income

membuat kategori usia nantinya digunakan untuk membuat kolom baru usia kategori yang fungsinya untuk membantu mencari nilai total_income yang hilang

In [32]:
def agegroup(age):
    try:
        if age <=30:
            return '<30'
        if  31 <= age <= 40:
            return '31 - 40'
        if 41 <= age  <= 50:
            return '41 - 50'
        if 51 <= age  <= 60:
            return '51 - 60'
        if 61 <=age  <= 70:
            return '61 - 70'
        else:
            return '>70'
    except:
        return 0
        

In [33]:
agegroup(90)

'>70'

In [34]:
#membuat kolom baru untuk kategori usia dalam data nantinya digunakan untuk membuat perkiraan total income dengan median dan mean
data['category_age'] = data['dob_years'].apply(agegroup)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_age
0,1,23.116912,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41 - 50
1,1,11.026860,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31 - 40
2,0,15.406637,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,31 - 40
3,3,11.300677,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,31 - 40
5,0,2.537495,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,<30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,6.442279,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate,31 - 40
21520,1,12.409087,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,41 - 50
21522,1,5.789991,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,31 - 40
21523,3,8.527347,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,31 - 40


Faktor yang bergantung pada pendapatan biasanya bergantung juga pada kelompok usia, biasanya jika usia sudah lebih tua maka pengalamannya lebih banyak di suatu bidang pekerjaan dan pendapatannya lebih besar dibanding yang umurnya lebih muda, faktor lain yang mempengaruhi mungkin dari segi pendidikan, semakin tinggi pendidikan maka tingkat pendapatan semakin besar pul

### melihat rata-rata gaji berdasarkan umur dan pekerjaan

In [35]:
#menggunakan data tanpa days_employee not null
data1 = data[data['days_employed'].notnull()]

In [36]:
by_mean = data1.pivot_table(index = 'category_age',columns='education',values = 'total_income',aggfunc='mean')
by_mean

education,bachelor's degree,graduate degree,primary education,secondary education,some college
category_age,,,,,
31 - 40,34229.473804,18187.3015,21708.247957,25934.200866,32091.097338
41 - 50,35827.844866,31771.3210,24671.047205,26084.708772,34824.451022
51 - 60,37158.917122,42945.7940,18816.059850,25525.630086,31307.589576
61 - 70,35374.296009,NaN,23657.521167,27052.430809,32149.037000
<30,29971.703696,NaN,26234.023138,23548.531695,25705.938862
>70,34610.091000,NaN,NaN,23839.259400,NaN


melihat rata rata gaji berdasarkan category age dan education

In [63]:
by_median = data1.pivot_table(index = 'category_age',columns='education',values ='total_income',aggfunc='median')
by_median.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 31 - 40 to >70
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   bachelor's degree    6 non-null      float64
 1   graduate degree      3 non-null      float64
 2   primary education    5 non-null      float64
 3   secondary education  6 non-null      float64
 4   some college         5 non-null      float64
dtypes: float64(5)
memory usage: 288.0+ bytes


melihat median gaji berdasarkan category age dan education

In [38]:
#membuat fungsi yang digunakan untuk mengisi nilai null berdasarkan mean
def get_mean_income(category_age,education):   
    try:
        return by_mean[education][category_age]
    except:
        return "error"

In [64]:
get_mean_income('41 - 50','secondary education')

26084.70877193501

In [ ]:
for 

In [40]:
data['mean_income'] = data.apply(lambda x : get_mean_income(x.category_age,x.education),axis=1)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_age,mean_income
0,1,23.116912,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41 - 50,35827.844866
1,1,11.026860,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31 - 40,25934.200866
2,0,15.406637,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,31 - 40,25934.200866
3,3,11.300677,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,31 - 40,25934.200866
5,0,2.537495,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,<30,29971.703696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,6.442279,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate,31 - 40,18187.301500
21520,1,12.409087,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,41 - 50,26084.708772
21522,1,5.789991,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,31 - 40,25934.200866
21523,3,8.527347,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,31 - 40,25934.200866


In [41]:
def get_median_income(category_age,education):
    try:
        return by_median[education][category_age]
    except:
        return 'error'

In [42]:
data['median_income'] = data.apply(lambda x: get_median_income(x.category_age,x.education),axis = 1)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_age,mean_income,median_income
0,1,23.116912,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41 - 50,35827.844866,30136.2360
1,1,11.026860,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31 - 40,25934.200866,23125.3060
2,0,15.406637,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,31 - 40,25934.200866,23125.3060
3,3,11.300677,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,31 - 40,25934.200866,23125.3060
5,0,2.537495,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,<30,29971.703696,26099.6350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,6.442279,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate,31 - 40,18187.301500,18187.3015
21520,1,12.409087,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,41 - 50,26084.708772,22858.9880
21522,1,5.789991,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,31 - 40,25934.200866,23125.3060
21523,3,8.527347,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,31 - 40,25934.200866,23125.3060


kita bisa mengganti total income yang nilainya NaN tadi dengan rata-rata atau median income yang didapatkan berdasarkan umur dan lama bekerja

In [43]:
#data[data['total_income'].isna()]
data['total_income'] = data['total_income'].fillna(data['median_income'])

saya mengisi nilai total income yang hilang berdasarkam median income yang didapat dari median total income yang dicari berdasarkan kelompok umur dan pendidikan

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17884 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          17884 non-null  int64  
 1   days_employed     15794 non-null  float64
 2   dob_years         17884 non-null  int64  
 3   education         17884 non-null  object 
 4   education_id      17884 non-null  int64  
 5   family_status     17884 non-null  object 
 6   family_status_id  17884 non-null  int64  
 7   gender            17884 non-null  object 
 8   income_type       17884 non-null  object 
 9   debt              17884 non-null  int64  
 10  total_income      17884 non-null  float64
 11  purpose           17884 non-null  object 
 12  category_age      17884 non-null  object 
 13  mean_income       17884 non-null  float64
 14  median_income     17884 non-null  float64
dtypes: float64(4), int64(5), object(6)
memory usage: 2.2+ MB


### mengisi nilai days_employed

In [66]:
#mencari rata-rata berdasarkan dob_years, biasanya pekerja yang umur tertentu bedanya tidak jauh
days_employed_mean = data1.pivot_table(index='dob_years',values='days_employed',aggfunc='mean')

In [46]:
#mencari median berdasarkan dob_years, biasanya pekerja yang umur tertentu bedanya tidak jauh
days_employed_median = data1.pivot_table(index='dob_years',values='days_employed',aggfunc='median')

In [47]:
def fill_days_employed(dob_years):
    try:
        return days_employed_median['days_employed'][dob_years]
    except:
        return 'error'

In [48]:
fill_days_employed(19)

1.9849112615664273

In [49]:
data['employed_median'] = data.apply(lambda x : fill_days_employed(x.dob_years),axis=1)
data['days_employed'] = data['days_employed'].fillna(data['employed_median'])

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17884 entries, 0 to 21524
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          17884 non-null  int64  
 1   days_employed     17884 non-null  float64
 2   dob_years         17884 non-null  int64  
 3   education         17884 non-null  object 
 4   education_id      17884 non-null  int64  
 5   family_status     17884 non-null  object 
 6   family_status_id  17884 non-null  int64  
 7   gender            17884 non-null  object 
 8   income_type       17884 non-null  object 
 9   debt              17884 non-null  int64  
 10  total_income      17884 non-null  float64
 11  purpose           17884 non-null  object 
 12  category_age      17884 non-null  object 
 13  mean_income       17884 non-null  float64
 14  median_income     17884 non-null  float64
 15  employed_median   17884 non-null  float64
dtypes: float64(5), int64(5), object(6)
memor

sudah tidak ada data yang bernilai null

## Pengkategorian Data

In [51]:
#melihat tujuan nasabah meminjam uang
data['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'housing transactions', 'education', 'having a wedding',
       'purchase of the house for my family', 'buy real estate',
       'buy commercial real estate', 'to have a wedding',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying my own car',
       'buying a second-hand car', 'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education',
       'transactions with commercial real estate', 'to own a car',
       'purchase of a car', 'profile education', 'university education',
       'to buy a car', 'buying property for renting out',
       'housing renovation', 'going

In [52]:
dictionary_1 = {'purchase of the house' : 'house',
                'car purchase' : 'car',
                'having a wedding' : 'wedding',
               }

In [53]:
data['category_purpose'] = data['purpose'].replace(dictionary_1)

In [54]:
def income_group(total_income):
    try:
        if (total_income >0) & (total_income<=10000):
            return 'low income'
        if (total_income >10000) & (total_income <=20000):
            return 'middle income'
        if (total_income >20000):
            return 'high income'
    except:
        return '0'

In [55]:
data['income_group'] = data['total_income'].apply(income_group)

### memeriksa hipotesis

In [56]:
#memeriksa apakah punya anak mempengaruhi gagal bayar atau tidak
childrendata = data.pivot_table(index='children',columns='debt',values='dob_years',aggfunc='count')
childrendata['gagal_bayar%'] = (childrendata[1]/(childrendata[1]+childrendata[0]))*100
childrendata

debt,0,1,gagal_bayar%
children,,,
0,9992.0,892.0,8.195516
1,4110.0,429.0,9.451421
2,1830.0,192.0,9.495549
3,296.0,26.0,8.074534
4,36.0,4.0,10.000000
5,9.0,NaN,NaN
20,60.0,8.0,11.764706


In [57]:
#memriksa apakah family status mempengaruhi gagal bayar atau tidak
familystatus = data.pivot_table(index='family_status',columns='debt',values='dob_years',aggfunc='count')
familystatus['gagal_bayar%'] = (familystatus[1]/(familystatus[0]+familystatus[1]))*100
familystatus

debt,0,1,gagal_bayar%
family_status,,,
civil partnership,3190,357,10.064844
divorced,911,74,7.512690
married,9570,828,7.963070
unmarried,2221,261,10.515713
widow / widower,441,31,6.567797


In [58]:
educationstatus = data.pivot_table(index='education',columns='debt',values='dob_years',aggfunc='count')
educationstatus['gagal_bayar%'] = (educationstatus[1]/(educationstatus[0]+educationstatus[1]))*100
educationstatus

debt,0,1,gagal_bayar%
education,,,
bachelor's degree,4436.0,254.0,5.415778
graduate degree,4.0,NaN,NaN
primary education,161.0,28.0,14.814815
secondary education,11094.0,1202.0,9.775537
some college,638.0,67.0,9.503546


In [59]:
income_types = data.pivot_table(index='income_type',columns='debt',values='dob_years',aggfunc='count')
income_types['gagal_bayar%'] = (income_types[1]/(income_types[1]+income_types[0]))*100
income_types

debt,0,1,gagal_bayar%
income_type,,,
business,4673.0,375.0,7.428685
civil servant,1361.0,86.0,5.943331
employee,9949.0,1054.0,9.579206
entrepreneur,2.0,NaN,NaN
paternity / maternity leave,NaN,1.0,NaN
retiree,347.0,35.0,9.162304
student,1.0,NaN,NaN


In [60]:
#tujuan meminjam uang berpengaruh pada gagal bayar
creditstatus = data.pivot_table(index='category_purpose',columns='debt',values='dob_years',aggfunc='count')
creditstatus['gagal_bayar%'] = (creditstatus[1]/(creditstatus[0]+creditstatus[1]))*100
creditstatus

debt,0,1,gagal_bayar%
category_purpose,,,
building a property,475,48,9.177820
building a real estate,463,44,8.678501
buy commercial real estate,514,40,7.220217
buy real estate,477,37,7.198444
buy residential real estate,477,39,7.558140
buying a second-hand car,361,32,8.142494
buying my own car,387,44,10.208817
buying property for renting out,500,46,8.424908
car,705,73,9.383033


### Kesimpulan

Kesimpulan yang didapat setelah melakukan analisis:

    1. berdasarkan jumlah anak: ada kaitan antara gagal bayar dengan mempunyai anak, nasabah yang mempunyai anak ada kemungkinan gagal bayar pinjaman walaupun tidak semua, dari data terlihat malah jumlah nasabah yang tidak memiliki anak malah paling banyak melakukan gagal bayar dengan 892 nasabah
    2. berdasarkan family status: ada kemungkinan family status mempengaruhi probabilitas gagal bayar dalam analisis data ini diketahui justru nasabah dengan status married paling banyak mengalami gagal bayar dibanding nasabah dengan status lainnya dengan total 828 nasabah, justru persentase gagal bayar status widow yang probabilitas gagal bayarnya paling kecil
    3. berdasarkan pendidikan : dari data education yang didapat ternyata probabilitas gagal bayar paling besar adalah nasabah yang pendidikannya secondary education dengan 1202 nasabah
    4. berdasarkan income_type : income_type ini berhubungan dengan pekerjaan atau profesi, jadi profesi employee disini yang persentase gagal bayar yang paling besar dibanding yang lainnya dengan 1054 nasabah
    5. rata rata tujuan untuk meminjam uang adalah untuk menikah, membeli rumah, dan membeli kendaraan (disini mobil)
    